In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("Customer360Insights.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   SessionStart           2000 non-null   object
 1   CustomerID             2000 non-null   int64 
 2   FullName               2000 non-null   object
 3   Gender                 2000 non-null   object
 4   Age                    2000 non-null   int64 
 5   CreditScore            2000 non-null   int64 
 6   MonthlyIncome          2000 non-null   int64 
 7   Country                2000 non-null   object
 8   State                  2000 non-null   object
 9   City                   2000 non-null   object
 10  Category               2000 non-null   object
 11  Product                2000 non-null   object
 12  Cost                   2000 non-null   int64 
 13  Price                  2000 non-null   int64 
 14  Quantity               2000 non-null   int64 
 15  CampaignSchema       

In [3]:
df = df.dropna(subset=['OrderConfirmationTime'])
df['OrderConfirmationTime'] = pd.to_datetime(df['OrderConfirmationTime'], format = '%Y-%m-%d')

In [4]:
cust = len(df['CustomerID'].value_counts())
print("Total number of distinct customers:", cust)

Total number of distinct customers: 1103


In [5]:
# Customer features that appear most relevant for clustering/segmentation (demographics and general information)
features = ['CustomerID', 'FullName', 'Gender', 'Age', 'CreditScore', 'MonthlyIncome', 'Country']


In [6]:
# Calculating Customer Recency
# Groups the customers by their details and their most recent purchase date
cust_df = df.groupby(by=features, as_index = False)['OrderConfirmationTime'].max()
recent_date = cust_df['OrderConfirmationTime'].max()

cust_df['Recency'] = cust_df['OrderConfirmationTime'].apply( lambda x: (recent_date - x).days)
cust_df.head()

,CustomerID,FullName,Gender,Age,CreditScore,MonthlyIncome,Country,OrderConfirmationTime,Recency
0,1001,Brittany Franklin,Male,57,780,7591,China,2022-12-23 18:32:00,372
1,1002,Scott Stewart,Female,69,746,3912,China,2023-09-27 22:02:00,94
2,1003,Elizabeth Fowler,Female,21,772,7460,UK,2019-01-04 03:40:00,1822
3,1004,Julian Wall,Female,67,631,4765,UK,2022-07-27 13:03:00,521
4,1005,James Simmons,Male,57,630,3268,China,2019-01-05 13:52:00,1820


In [7]:
# Calculating Customer frequency
df_freq = df.groupby(by=['CustomerID', 'FullName'], as_index = False)['OrderConfirmationTime'].count()
cust_df['Frequency'] = df_freq['OrderConfirmationTime']
cust_df.head()

,CustomerID,FullName,Gender,Age,CreditScore,MonthlyIncome,Country,OrderConfirmationTime,Recency,Frequency
0,1001,Brittany Franklin,Male,57,780,7591,China,2022-12-23 18:32:00,372,2
1,1002,Scott Stewart,Female,69,746,3912,China,2023-09-27 22:02:00,94,2
2,1003,Elizabeth Fowler,Female,21,772,7460,UK,2019-01-04 03:40:00,1822,1
3,1004,Julian Wall,Female,67,631,4765,UK,2022-07-27 13:03:00,521,2
4,1005,James Simmons,Male,57,630,3268,China,2019-01-05 13:52:00,1820,1


In [8]:
# Revenue
df['Revenue'] = df['Price'] * df['Quantity']

# Calculating total spent per customer
df_tspent = df.groupby(by=['CustomerID', 'FullName'], as_index = False)['Revenue'].sum()
cust_df['Monetary Value'] = df_tspent['Revenue']
cust_df.head()

,CustomerID,FullName,Gender,Age,CreditScore,MonthlyIncome,Country,OrderConfirmationTime,Recency,Frequency,Monetary Value
0,1001,Brittany Franklin,Male,57,780,7591,China,2022-12-23 18:32:00,372,2,272
1,1002,Scott Stewart,Female,69,746,3912,China,2023-09-27 22:02:00,94,2,1080
2,1003,Elizabeth Fowler,Female,21,772,7460,UK,2019-01-04 03:40:00,1822,1,40
3,1004,Julian Wall,Female,67,631,4765,UK,2022-07-27 13:03:00,521,2,2040
4,1005,James Simmons,Male,57,630,3268,China,2019-01-05 13:52:00,1820,1,600


In [9]:
#Compute numerical data ranks (1 through n) along axis. By default, equal values are assigned a rank that is the average of the ranks of those values.
cust_df['R_rank'] = cust_df['Recency'].rank(ascending=False) # Lower values get assigned higher ranks
cust_df['F_rank'] = cust_df['Frequency'].rank(ascending=True) # Higher values get assigned higher ranks
cust_df['M_rank'] = cust_df['Monetary Value'].rank(ascending=True) # Higher values get assigned higher ranks

# Normalizing ranks, diving by the highest rank and multiplying the result with 100
cust_df['R_rank_norm'] = (cust_df['R_rank']/cust_df['R_rank'].max())*100
cust_df['F_rank_norm'] = (cust_df['F_rank']/cust_df['F_rank'].max())*100
cust_df['M_rank_norm'] = (cust_df['F_rank']/cust_df['M_rank'].max())*100
 
cust_df.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)

In [10]:
cust_df['RFM_Score'] = (0.15*cust_df['R_rank_norm']) + (0.28 * cust_df['F_rank_norm']) + (0.57*cust_df['M_rank_norm'])
cust_df['RFM_Score'] *= 0.05
cust_df = cust_df.round(2)
cust_df.head(10)

,CustomerID,FullName,Gender,Age,CreditScore,MonthlyIncome,Country,OrderConfirmationTime,Recency,Frequency,Monetary Value,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score
0,1001,Brittany Franklin,Male,57,780,7591,China,2022-12-23 18:32:00,372,2,272,72.30,74.05,73.62,3.68
1,1002,Scott Stewart,Female,69,746,3912,China,2023-09-27 22:02:00,94,2,1080,92.75,74.05,73.62,3.83
2,1003,Elizabeth Fowler,Female,21,772,7460,UK,2019-01-04 03:40:00,1822,1,40,0.09,29.18,29.01,1.24
3,1004,Julian Wall,Female,67,631,4765,UK,2022-07-27 13:03:00,521,2,2040,63.15,74.05,73.62,3.61
4,1005,James Simmons,Male,57,630,3268,China,2019-01-05 13:52:00,1820,1,600,0.18,29.18,29.01,1.24
5,1006,Annette Atkins,Female,36,660,5350,India,2022-10-20 04:11:00,437,3,249,68.09,94.57,94.02,4.51
6,1007,Martha Smith,Female,47,606,3972,UK,2019-01-08 23:20:00,1817,1,500,0.27,29.18,29.01,1.24
7,1008,Kristina Ramirez,Female,45,627,7455,India,2022-11-13 22:55:00,412,2,1605,69.72,74.05,73.62,3.66
8,1009,Justin Hardin,Male,59,609,3003,Spain,2019-01-09 16:58:00,1816,1,80,0.36,29.18,29.01,1.24
9,1011,Zachary Roberts,Male,60,634,6753,USA,2023-09-03 21:03:00,118,2,170,91.02,74.05,73.62,3.82


In [12]:
# rfm score >= 4.5 : Top Customer
# 4.5 > rfm score >= 4 : High Value Customer
# 4 > rfm score >= 3 : Medium Value customer
# 3 > rfm score >= 1.5 : Low Value customer
# rfm score < 1.5 : Lost Customer

def segment(x):
    if x >= 4.5:
        return "Top Customer"
    elif x < 4.5 and x >= 4:
        return "High Value Customer"
    elif x < 4 and x >= 3:
        return "Medium Value customer"
    elif x < 3 and x >= 1.5:
        return "Low Value customer"
    else:
        return "Lost Customer"

cust_df['Customer Segment'] = cust_df['RFM_Score'].apply(segment)
cust_df

,CustomerID,FullName,Gender,Age,CreditScore,MonthlyIncome,Country,OrderConfirmationTime,Recency,Frequency,Monetary Value,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer Segment
0,1001,Brittany Franklin,Male,57,780,7591,China,2022-12-23 18:32:00,372,2,272,72.30,74.05,73.62,3.68,Medium Value customer
1,1002,Scott Stewart,Female,69,746,3912,China,2023-09-27 22:02:00,94,2,1080,92.75,74.05,73.62,3.83,Medium Value customer
2,1003,Elizabeth Fowler,Female,21,772,7460,UK,2019-01-04 03:40:00,1822,1,40,0.09,29.18,29.01,1.24,Lost Customer
3,1004,Julian Wall,Female,67,631,4765,UK,2022-07-27 13:03:00,521,2,2040,63.15,74.05,73.62,3.61,Medium Value customer
4,1005,James Simmons,Male,57,630,3268,China,2019-01-05 13:52:00,1820,1,600,0.18,29.18,29.01,1.24,Lost Customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2196,Regina Kidd,Female,21,655,5220,China,2022-09-14 15:34:00,472,2,1675,65.64,74.05,73.62,3.63,Medium Value customer
1099,2197,Mr. Matthew Perez MD,Female,35,613,4948,Canada,2023-12-22 17:00:00,8,3,300,99.64,94.57,94.02,4.75,Top Customer
1100,2198,Sarah Howard,Male,36,637,5856,China,2023-08-25 08:49:00,127,2,250,90.21,74.05,73.62,3.81,Medium Value customer
1101,2199,Steven Larsen,Female,31,619,3495,China,2021-12-30 06:00:00,731,1,50,52.77,29.18,29.01,1.63,Low Value customer


In [13]:
cust_df.to_csv("customer_segmentation.csv", index=False)